In [ ]:
from transformers import AutoTokenizer, TapasForQuestionAnswering
import pandas as pd

# Define the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("google/tapas-base-finetuned-wtq")
model = TapasForQuestionAnswering.from_pretrained("google/tapas-base-finetuned-wtq")

# Define the data for the table
data = {
    "Company": ["Apple", "Microsoft", "Google"],
    "CEO": ["Tim Cook", "Satya Nadella", "Sundar Pichai"],
    "Headquarters": ["Cupertino", "Redmond", "Mountain View"]
}
# Convert the data into a pandas DataFrame
table = pd.DataFrame.from_dict(data)
# Define the questions (queries)
queries = ["Who is the CEO of Microsoft?", "Where is the headquarters of Google?"]

# Tokenize the table and queries
inputs = tokenizer(table=table, queries=queries, padding="max_length", return_tensors="pt")
# Make predictions with the model
outputs = model(**inputs)

# Extract the predicted answer coordinates and aggregation indices
predicted_answer_coordinates, predicted_aggregation_indices = tokenizer.convert_logits_to_predictions(
    inputs,
    outputs.logits.detach(),
    outputs.logits_aggregation.detach()
)

# Iterate over the queries and print the answers
for i, query in enumerate(queries):
    if predicted_aggregation_indices[i] == 0:
        # If there is no aggregation operation (index 0), print the cells
        coords_to_answer = ' '.join([table.iat[coord] for coord in predicted_answer_coordinates[i]])
        print(f"Question: {query}")
        print(f"Answer: {coords_to_answer}\n")
    else:
        # If there is an aggregation operation, print the operation's name (from the list of operations)
        print(f"Question: {query}")
        print(f"Answer: {tokenizer.model.config.id2label[predicted_aggregation_indices[i]]}\n")


Question: Who is the CEO of Microsoft?
Answer: Satya Nadella

Question: Where is the headquarters of Google?
Answer: Mountain View

